In [1]:
import numpy as np
import gmsh

gmsh.initialize() #inicia Gmsh
gmsh.model.add('PROBLEMA2') #genera el modelo


# Generado de la figura
L1 = 10
L2 = 5
r = 1
#Lc = 20 #caso del ejercicio
Lc = 1
Lcr = 1  #Refina aun mas el mallado cerca del circulo

#chapa
p1 = gmsh.model.geo.addPoint(0, 0, 0, Lc)
p2 = gmsh.model.geo.addPoint(0, L2, 0, Lc) 
p3 = gmsh.model.geo.addPoint(L1, L2, 0, Lc) 
p4 = gmsh.model.geo.addPoint(L1, 0, 0, Lc)

#circulo
p5 = gmsh.model.geo.addPoint(0,r,0,Lcr)
p6 = gmsh.model.geo.addPoint(r,0,0,Lcr)


l1 = gmsh.model.geo.addLine(p5, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p6)

semi1 = gmsh.model.geo.addCircleArc(p5,p1 ,p6) 


C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
C2 = gmsh.model.geo.addCurveLoop([semi1])

S1 = gmsh.model.geo.addPlaneSurface([C1,C2]) #asi se genera la superifice descontando el circulo


#Generado de las condiciones de contorno

Empotradox = gmsh.model.addPhysicalGroup(1, [l1]) #1 es para curva o linea!
gmsh.model.setPhysicalName(1,Empotradox,'Empotradox')


Empotradoy = gmsh.model.addPhysicalGroup(1, [l4]) #1 es para curva o linea!
gmsh.model.setPhysicalName(1,Empotradoy,'Empotradoy')


Traccionado = gmsh.model.addPhysicalGroup(1, [l3])
gmsh.model.setPhysicalName(1,Traccionado,'Traccionado') #esta funcion le pone nombre a todos los nodos aca


Superficie = gmsh.model.addPhysicalGroup(2,[S1]) #2 es para superficies!
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')


gmsh.model.geo.synchronize() #sincroniza lo que creamos con el modelo que creamos arriba
gmsh.model.mesh.generate(2)

In [2]:
NodeInfo = gmsh.model.mesh.get_nodes()
NodeInfo[1].shape
NumeroNodos = NodeInfo[0].shape[0]
MN = NodeInfo[1].reshape(NumeroNodos , 3)


ElementInfo = gmsh.model.mesh.get_elements()
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)
MC = ELEMENTS.reshape([ETAGS.shape[0],3])

#Empotrados
NodosIzquierda = gmsh.model.mesh.get_nodes_for_physical_group(1,Empotradox)
A = NodosIzquierda[0].reshape(-1,1)
B = np.zeros((len(A),1))
MNEx = np.concatenate((A,B) , axis = 1)

NodosAbajo = gmsh.model.mesh.get_nodes_for_physical_group(1,Empotradoy)
J = NodosAbajo[0].reshape(-1,1)
T = np.zeros((len(J),1))
MNEy = np.concatenate((J,T) , axis = 1)


MNE = np.concatenate((MNEx,MNEy) , axis = 0)

#Traccionados

In [3]:
'''ya de entrada aca hay algo mal, la ultima longitud pese a ser un mallado sietrico es de 3 y no de 1 por lo tanto las fuerzas estan mal'''

entityTraccionada = gmsh.model.getEntitiesForPhysicalGroup(1, Traccionado)
Tgroup, Ttraccionada, Ltraccionada = gmsh.model.mesh.getElements(1, entityTraccionada[0])
Ltraccionada = Ltraccionada[0].reshape(Ttraccionada[0].shape[0],2)

In [4]:
Ltraccionada

array([[ 2, 18],
       [18, 19],
       [19, 20],
       [20, 21],
       [21,  3]], dtype=uint64)

In [5]:
Ttraccionada

[array([5, 6, 7, 8, 9], dtype=uint64)]

In [6]:
gmsh.fltk.run()

In [7]:
Longitudes = np.abs( MN[Ltraccionada[:,0],1] - MN[Ltraccionada[:,1],1] )

In [8]:
NodeInfo[0]

array([80,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79], dtype=uint64)

In [9]:
Longitudes

array([1., 1., 1., 1., 1.])

In [10]:
Longitudesesor = 1
tension = 1000 #Pa

NodosTraccionados = gmsh.model.mesh.get_nodes_for_physical_group(1, Traccionado)
# MDF_COMMENT fijate que ya los tenes en Ltraccionada !

In [11]:
Ltraccionada

array([[ 2, 18],
       [18, 19],
       [19, 20],
       [20, 21],
       [21,  3]], dtype=uint64)

In [12]:
MDFFuerzas = np.zeros((2*NumeroNodos,1))

In [13]:
MDFDeslaza = np.zeros((2*NumeroNodos,1))

In [14]:
espesor = 1

In [15]:
NumeroNodos

80

In [16]:
for l, linea in enumerate(Ltraccionada):
    n1, n2 = linea
#    longitud = np.abs( MN[n1,1] - MN[n2,1])
#    Flocal = tension*espesor*longitud*1/2 
    Flocal = Longitudes[l]*tension*espesor/2
    MDFFuerzas[int(2*n1),0] += Flocal
    MDFFuerzas[int(2*n2),0] += Flocal

In [17]:
Fx = MDFFuerzas[:-1:2]
Fy = MDFFuerzas[1::2]

In [18]:
Fxyz = np.hstack([Fx,Fy,np.zeros_like(Fy)])

In [20]:
FEx = gmsh.view.add("Fuerzas Externas")
Fexterna = gmsh.view.addModelData(
    FEx,
    0,
    'PROBLEMA2',
    'NodeData', NodeInfo[0],
    Fxyz , numComponents=3)
#agrega todos los datos , el 0 viene de dependencias temporarles aca es 0 no varia con t



In [24]:
gmsh.fltk.run()

In [29]:
MEx = gmsh.model.mesh.get_nodes_for_physical_group(1, Empotradox)[0]
MEy = gmsh.model.mesh.get_nodes_for_physical_group(1, Empotradoy)[0]

In [30]:
MEy

array([ 3,  5, 22, 23, 24, 25, 26, 27, 28, 29], dtype=uint64)

In [31]:
2*MEx

array([ 2,  8, 12, 14, 16], dtype=uint64)

In [33]:
2*MEy + 1

array([ 7, 11, 45, 47, 49, 51, 53, 55, 57, 59], dtype=uint64)

In [35]:
s = np.hstack([2*MEx , 2*MEy+1])

In [36]:
s

array([ 2,  8, 12, 14, 16,  7, 11, 45, 47, 49, 51, 53, 55, 57, 59],
      dtype=uint64)

In [42]:
r = np.delete( np.arange(2*NumeroNodos) , s) 

In [43]:
r

array([  0,   1,   3,   4,   5,   6,   9,  10,  13,  15,  17,  18,  19,
        20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
        33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        48,  50,  52,  54,  56,  58,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159])

In [40]:
r = []
for i in range(2*NumeroNodos):
    if i not in s:
        r.append(i)
r = np.array(r, dtype = int)

In [41]:
r

array([  0,   1,   3,   4,   5,   6,   9,  10,  13,  15,  17,  18,  19,
        20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
        33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        48,  50,  52,  54,  56,  58,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159])

In [21]:
C = NodosTraccionados[0].reshape(-1,1)
D = np.zeros((len(C),2))
MNT = np.concatenate((C,D) , axis = 1)
for l in range(len(Longitudes)):
    MNT[l,1] = tension*espesor*Longitudes[l]
    
MNT[0,1] = tension*espesor*Longitudes[1]*1/2
MNT[len(C)-1,1] = tension*espesor*Longitudes[len(Longitudes)-1]*1/2
print(MNT)

Nodoslibres = []


'''la idea aca es tomar todos los nodos del centro asignarles fuerza 0 y agregegarlos a MNT incluidos los nodos empotrados ya que luego los discriminare cuando arme el vector fuerzas'''
Nodos = gmsh.model.mesh.get_nodes_for_physical_group(2, Superficie)
for i in Nodos[0]:
    if i in NodosTraccionados[0]:
        l = 0
    else:
        Nodoslibres.append(i)
        
E = np.array(Nodoslibres).reshape(-1,1)
D = np.zeros((len(E),2))
Interior = np.concatenate((E,D) , axis = 1)
MNT = np.concatenate((Interior,MNT) , axis = 0)

[[   2.  500.    0.]
 [   3. 1000.    0.]
 [  18. 1000.    0.]
 [  19. 1000.    0.]
 [  20. 1000.    0.]
 [  21.  500.    0.]]


In [22]:
gmsh.model.geo.synchronize()

In [23]:
gmsh.fltk.run()

In [12]:
E = 30E6
v = 0.3

D = (E/(1-v**2))*np.array([[1,v,0],[v,1,0],[0,0,(0.5*(1-v))]])
Matriz_coordenadas = np.ones((3,3))
Matriz_global = np.zeros([2*len(MN),2*len(MN)])
Matrices_Bloc = []
Areas = []

In [13]:
for l in range (len(MC)):
    
    I,M,J = MC[l]  
    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)
    
    x = np.array([MN[i,0],MN[m,0],MN[j,0]])
    y = np.array([MN[i,1],MN[m,1],MN[j,1]])

    Bi = y[1] - y[2] ;   Bm = y[2] - y[0] ;    Bj = y[0] - y[1]
    Gi = x[2] - x[1] ;   Gm = x[0] - x[2] ;    Gj = x[1] - x[0]      
    Matriz_coordenadas[:,0] = x
    Matriz_coordenadas[:,1] = y
    
    
    Area = np.linalg.det(Matriz_coordenadas)/2
    
    Matriz1 = (1/(2*Area)) * np.array([[Bi,0,Bm,0,Bj,0],[0,Gi,0,Gm,0,Gj],[Gi,Bi,Gm,Bm,Gj,Bj]])
    Matrices_Bloc.append(Matriz1)
    Matriz_local = espesor * abs(Area) * (np.dot(np.transpose(Matriz1), np.dot(D,Matriz1)))

    A_G = np.array([i*2,i*2+1,m*2,m*2+1,j*2,j*2+1]) #aca pasa algo raro
    Matriz_global[np.ix_(A_G,A_G)] += Matriz_local
    

In [15]:
Desplazamientos_f = np.zeros(len(MN)*2)
Fuerza = []
Nodos = []

for i in MNE:
    
    Nodo,valor = i 
    nodo = int(Nodo-1)
    
    if Nodo in NodosIzquierda[0]:
        Desplazamientos_f[2*nodo] = valor # '''valor en x es nulo'''

    elif Nodo in NodosAbajo[0]:
        Desplazamientos_f[2*nodo+1] = valor #'''valor en y es nulo'''

#'''Aca armo el vector fuerzas, si el nodo es alguno empotrado solo tomo el valor x o y respectivamente y le asigno un valor igual a 0'''
for i in MNT:
    Nodo,valorx,valory = i
    nodo = int(Nodo-1)
    
    if Nodo in NodosIzquierda[0]:
        Fuerza.append(0)
        Nodos.append(2*nodo+1) # para los nodos a la izquierda la fuerza en y es 0
    elif Nodo in NodosAbajo[0]:
        Fuerza.append(0)
        Nodos.append(2*nodo)   # para los nodos abajo la fuerza en x es nula
    else:
        Fuerza.append(valorx)
        Fuerza.append(valory)
        Nodos.append(2*nodo)
        Nodos.append(2*nodo+1) 

    
Fuerza = np.array(Fuerza).reshape(-1,1)

Matriz_res = Matriz_global[np.ix_(Nodos,Nodos)]

Desplazamientos_parcial = np.linalg.solve(Matriz_res,Fuerza)


o = 0
for i in Nodos:
        Desplazamientos_f[i] = Desplazamientos_parcial [o]
        o += 1


Fuerzas_ = np.dot(Matriz_global,Desplazamientos_f)
#print(Fuerzas_.round(4))
#print()
#
#print(Desplazamientos_f)

Me cuesta seguir que cosa es cada cosa pero me parece que te hiciste lio. vos tenes los nodos traccionados, 

In [20]:
Ltraccionada

array([[ 2, 18],
       [18, 19],
       [19, 20],
       [20, 21],
       [21,  3]], dtype=uint64)

In [16]:
#Esfuerzos

Tensiones = []

for p in range(len(MC)):
    
    I,M,J = MC[p]    
    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)

    
    nodos = [2*i,2*i+1,2*m,2*m+1,2*j,2*j+1]
    
    
    A = np.dot(Matrices_Bloc[p],Desplazamientos_f[nodos])
    
    B = np.dot(D,A)

   
    Tensiones.append(B)

Tensiones_ = np.array(Tensiones)

# for i in range (len(Tensiones)):
#    print(Tensiones[i])
#    print()

In [17]:
Problema2 = gmsh.view.add("Problema2")

problema2 = gmsh.view.addModelData(Problema2, 0, 'PROBLEMA2', 'ElementData', ETAGS, Tensiones_[:,0].reshape(-1,1) , numComponents=1)
#agrega todos los datos , el 0 viene de dependencias temporarles aca es 0 no varia con t



In [18]:
gmsh.fltk.run()

In [8]:
Desplazamientos = np.zeros((len(MN),3))

for i in range(len(MN)):
    Desplazamientos[i] = [Desplazamientos_f[i*2],Desplazamientos_f[i*2+1],0]

Problema12 = gmsh.view.add("Problema2")

problema12 = gmsh.view.addModelData(Problema12, 0, 'PROBLEMA2', 'NodeData', NodeInfo[0], Desplazamientos  , numComponents=3)
#agrega todos los datos , el 0 viene de dependencias temporarles aca es 0 no varia con t



In [ ]:
gmsh.fltk.run() 